<a href="https://colab.research.google.com/github/mikael-daniels/Datasets/blob/master/recurrent_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
starbucks = pd.read_csv('sbux.csv')
starbucks.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,27.920,28.325,27.920,28.185,7146296,SBUX
1,2013-02-11,28.260,28.260,27.930,28.070,5457354,SBUX
2,2013-02-12,28.000,28.275,27.975,28.130,8665592,SBUX
3,2013-02-13,28.230,28.230,27.750,27.915,7022056,SBUX
4,2013-02-14,27.765,27.905,27.675,27.775,8899188,SBUX


In [3]:
starbucks['return'] = starbucks['close'].shift(1)
starbucks.head()

,date,open,high,low,close,volume,Name,return
0,2013-02-08,27.920,28.325,27.920,28.185,7146296,SBUX,NaN
1,2013-02-11,28.260,28.260,27.930,28.070,5457354,SBUX,28.185
2,2013-02-12,28.000,28.275,27.975,28.130,8665592,SBUX,28.070
3,2013-02-13,28.230,28.230,27.750,27.915,7022056,SBUX,28.130
4,2013-02-14,27.765,27.905,27.675,27.775,8899188,SBUX,27.915


In [4]:
starbucks['return'] = (starbucks['close']-starbucks['return'])/starbucks['return']
starbucks.head()

,date,open,high,low,close,volume,Name,return
0,2013-02-08,27.920,28.325,27.920,28.185,7146296,SBUX,NaN
1,2013-02-11,28.260,28.260,27.930,28.070,5457354,SBUX,-0.004080
2,2013-02-12,28.000,28.275,27.975,28.130,8665592,SBUX,0.002138
3,2013-02-13,28.230,28.230,27.750,27.915,7022056,SBUX,-0.007643
4,2013-02-14,27.765,27.905,27.675,27.775,8899188,SBUX,-0.005015


In [6]:
features = starbucks.iloc[:, 1:6]
features.head()

,open,high,low,close,volume
0,27.920,28.325,27.920,28.185,7146296
1,28.260,28.260,27.930,28.070,5457354
2,28.000,28.275,27.975,28.130,8665592
3,28.230,28.230,27.750,27.915,7022056
4,27.765,27.905,27.675,27.775,8899188


In [7]:
target = starbucks['return']
target.head()

0         NaN
1   -0.004080
2    0.002138
3   -0.007643
4   -0.005015
Name: return, dtype: float64

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)
features

array([[0.03614772, 0.03955322, 0.04377637, 0.04161728, 0.09623221],
       [0.04502153, 0.03784494, 0.04404008, 0.03858817, 0.06327041],
       [0.03823568, 0.03823916, 0.04522679, 0.04016858, 0.12588316],
       ...,
       [0.75675323, 0.77371879, 0.74657173, 0.73976031, 0.27019334],
       [0.70859977, 0.76846255, 0.71993671, 0.76399315, 0.29664   ],
       [0.74500848, 0.75190539, 0.74314346, 0.73370209, 0.22856651]])

In [12]:
T = 10
N = len(starbucks) - T
D = 5

In [13]:
train_length = round(len(starbucks) * 0.65)
X_train = np.zeros((train_length, T, D))
y_train = np.zeros(train_length)

In [14]:
for t in range(train_length):
  X_train[t, :, :] = features[t:t+T]
  y_train[t] = target[t+T] > 0

In [15]:
X_test = np.zeros((N-train_length, T, D))
y_test = np.zeros(N-train_length)

In [16]:
for u in range(N-train_length):
  t = u + train_length
  X_test[u, :, :] = features[t:t+T]
  y_test[u] = target[t+T] > 0

In [23]:
model = Sequential()

model.add(LSTM(5, input_shape=(T, D)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5061
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 0.6926 - accuracy: 0.5269
Epoch 3/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6925 - accuracy: 0.5269
Epoch 4/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6926 - accuracy: 0.5269
Epoch 5/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6930 - accuracy: 0.5196
Epoch 6/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6923 - accuracy: 0.5269
Epoch 7/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.5269
Epoch 8/200
26/26 [==============================] - 0s 3ms/step - loss: 0.6924 - accuracy: 0.5183
Epoch 9/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6918 - accuracy: 0.5269
Epoch 10/200
26/26 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.5269
Epoch 11/

In [25]:
model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.6976 - accuracy: 0.5174


[0.6976437568664551, 0.517401397228241]